<a href="https://colab.research.google.com/github/tiger2221504/Prmn2023a/blob/main/LINEBot_opencv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 画像加工Bot
LINE Developers
https://developers.line.biz/console/

※最初にhaarcascade_frontalface_alt.xmlを読み込む。

In [1]:
# 最初に実行してください
!pip install line-bot-sdk
!pip install pyngrok
!pip install python-dotenv
!pip install pyimgur

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.1/729.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.27.1, but you have requests 2.31.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=632ceb455dbc43986245f02fba9180960888cdd544cab4d6113ddf82a14f0090
  Stored in directory: /root/.cache/pip/wheels/5

In [3]:
# 自分のトークンを設定してください
import os

directory = 'content/info'
if not os.path.exists(directory):
    os.makedirs(directory)

file_path = os.path.join(directory, '.env')
with open(file_path, 'w') as f:
    f.write('CHANNEL_ACCESS_TOKEN = "S+6QsiayZAEulY8jRhzWEScbR5ctMPUtfqSPvRiNvqxEQXY9y6NlZBYlfds3YnfaIhhWh7TgKpyw3tMGHvcmmqpK5M8YAWSDZTWxrMPk9bQaEHSwS+Y7S7albRXQSM3PdK0ZWpleC/c4o60w62IlRAdB04t89/1O/w1cDnyilFU="\n')
    f.write('CHANNEL_SECRET = "6a4dc8f36549946614607262615ceb0d"\n')

In [4]:
import os
from pathlib import Path
from datetime import datetime
import pytz

from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage, ImageMessage, ImageSendMessage
from pyngrok import ngrok
import dotenv

import cv2
import pyimgur

def mosaic(src, ratio=0.1):
    blur = cv2.blur(src, ksize=(50, 50))
    small = cv2.resize(blur, None, fx=ratio, fy=ratio, interpolation=cv2.INTER_NEAREST)
    return cv2.resize(small, src.shape[:2][::-1], interpolation=cv2.INTER_NEAREST)

def mosaic_area(src, x, y, width, height, ratio=0.1):
    dst = src.copy()
    dst[y:y + height, x:x + width] = mosaic(dst[y:y + height, x:x + width], ratio)
    return dst

ngrok.set_auth_token("2S3iDWMK7bFe4uOtFRVA8mWKpFo_2ztXxuGP15rSWH6HU6ET5")

app = Flask(__name__)
# LineBotApiオブジェクトを作成
dotenv.load_dotenv("content/info/.env")
line_bot_api = LineBotApi(os.getenv('CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(os.getenv('CHANNEL_SECRET'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)  # リクエストの署名検証を行い、正しければハンドラを実行
    except InvalidSignatureError:
        abort(400)  # 署名が無効な場合はエラーを返す
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    print("text:", event.message.text)  # 返受信したメッセージを表示

@handler.add(MessageEvent, message=ImageMessage)
def handle_image_message(event):
    message_id = event.message.id  # 受信した画像メッセージのIDを取得
    message_content = line_bot_api.get_message_content(message_id)  # 画像メッセージの内容を取得

    # 画像を保存する一時ファイルのパスを作成
    tmp_dir = "image"
    os.makedirs(tmp_dir, exist_ok=True)
    # タイムゾーンを設定
    tz = pytz.timezone('Asia/Tokyo')
    # 東京の現在の日時情報を取得
    now = datetime.now(tz)
    # 日時情報からファイル名を生成
    tmp_path = f"{tmp_dir}/{now.strftime('%Y%m%d%H%M%S')}.jpg"

    # 画像を一時ファイルとして保存
    with open(tmp_path, "wb") as f:
        for chunk in message_content.iter_content():
            f.write(chunk)

    # 画像の読み込み
    src = cv2.imread(tmp_path)
    dst_face = src.copy()

    # OpenCVが用意した顔認識モデルを読み込み
    face_cascade_path = 'haarcascade_frontalface_alt.xml'
    face_cascade = cv2.CascadeClassifier(face_cascade_path)

    # 画像のグレースケール化
    src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

    # 顔認識
    faces = face_cascade.detectMultiScale(src_gray)

    # 顔にモザイクをかける
    for x, y, w, h in faces:
        dst_face = mosaic_area(dst_face, x, y, w, h)

    cv2.imwrite(tmp_path, dst_face)

    # imgurにアップロード
    CLIENT_ID = "8509cd01a827f0b"
    image = pyimgur.Imgur(CLIENT_ID)
    uploaded_image = image.upload_image(tmp_path,title = "LINE_BOT")
    print(uploaded_image.link)

    # 画像を返す
    line_bot_api.reply_message(
        event.reply_token,
        ImageSendMessage(original_content_url=uploaded_image.link, preview_image_url=uploaded_image.link)
    )
    print("返信完了!!")  # 返信が完了したことを表示
    os.remove(tmp_path) # ファイル削除


ngrok_tunnel = ngrok.connect(5000)  # ポート5000でngrokのトンネルを作成
print('Public URL:', ngrok_tunnel.public_url)  # 公開されたURLを表示

if __name__ == "__main__":
    app.run()  # アプリケーションを実行

<ipython-input-4-6f083ef135c6>:31: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(os.getenv('CHANNEL_ACCESS_TOKEN'))
<ipython-input-4-6f083ef135c6>:32: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(os.getenv('CHANNEL_SECRET'))


Public URL: https://2b32-34-171-216-21.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [08/Jul/2023 13:33:36] "POST /callback HTTP/1.1" 200 -
<ipython-input-4-6f083ef135c6>:51: LineBotSdkDeprecatedIn30: Call to deprecated method get_message_content. (Use 'from linebot.v3.messaging import MessagingApiBlob' and 'MessagingApiBlob(...).get_message_content(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  message_content = line_bot_api.get_message_content(message_id)  # 画像メッセージの内容を取得


https://i.imgur.com/GlTej7y.jpg


<ipython-input-4-6f083ef135c6>:95: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(
INFO:werkzeug:127.0.0.1 - - [08/Jul/2023 13:33:55] "POST /callback HTTP/1.1" 200 -


返信完了!!


INFO:werkzeug:127.0.0.1 - - [08/Jul/2023 13:34:16] "POST /callback HTTP/1.1" 200 -


text: あああ
